<h1><center>Mapping the Seafloor</center></h1>

## Milestone Items
The following are required to complete the milestone for this exercise:
1. Submit your figure of the bathymetry along the dock
2. Submit the python code used to create your figure

Provide a 2-3 sentence response addressing the following milestone questions:
- Why might you have gotten inconsistent readings when taking measurements off of the dock?
- If you were to use an underwater echosounder in Puget Sound instead of Portage Bay, how and why might you expect the speed of sound to differ?

## Overview
For this activity, you will be using a **HCSR04** 40 kHz ultrasonic sensor to determine the bathymetry along the dock.  The HCSR04 sensor sends out a pulse of sound through one transducer, and listens for the echo return in the second transducer.  Based on the delay between sending the signal and receiving an echo back, you will be able to determine how far away an object is from the sensor.

This activity uses the waterproof transducer version of the HCSR04 ultrasonic acoustic sensor called a **JSN-SR04** to create a rough bathymetric chart of a region of off a pier or dock.  For this excercise you will collect measurements of the depth to the seafloor along the dock with 1 meter spacing.

## Bathymetry Measurements
You will be using the JSN-SR04 to collect measurements of depth at a series of fixed positions along the dock.

### Setting the Sound Speed
First, we need to define the sound speed of the water so we can get a correct measure from distance.  Recall from Part 1 that slight changes in sound speed affect the distance calculated.  We'll define sound speed as the following:


In [ ]:
soundSpeed = [1440] # a list of sound speeds, i.e., can be range() or [200]

### Prepare to Store the Data

Now let's get set up to take some measurements.  We're going to store the measurements in a python dictionary, so let's make an empty one:

In [ ]:
allSamples = {} # let's make an empty dictionary that we'll fill

### Set up the Sensor

Like in Part 1, we're going to have to tell the microcontroller where the acoustic sensor is plugged in, we we can define those two pins below:

In [ ]:
triggerPin = 12 #board pin number to 'Trig' on sensor
echoPin = 14 #board pin number to 'Echo' on sensor

To actually take the measurements, we're going to use `pySerial` like in Part 1.  The following function will be used to send those commands and return a measurement which will be stored in the dictionary we just created. Read through the function below so you understand what is being done, and run the cell so that we can use it. 

In [ ]:
def dist(triggerPin,echoPin,soundSpeed,site,allSamples): #We give the function the pins for the board, soundspeed, measurement site, and our dictionary
    for speed in soundSpeed: # for each sound speed (here, only for the 1 value)
        command = 'sensor = hcsr04v2.HCSR04(trigger_pin = '+str(triggerPin)+', echo_pin = '+str(echoPin)+', c = '+str(speed)+')\r' # set up the sensor on the microcontroller
        ser.write(bytes(command, 'utf-8')) # send the command
        ser.write(b'sensor.distance()\r') # take a distance measurement
        sleep(.5) # wait for .5 seconds for the sensor to respond
        input_data = ser.read(ser.inWaiting()) # read the response
        print('Command Sent: '+ str(input_data).split('\\r\\n>>')[0]) # print out what we sent
        print('Distance measured: ',float(str(input_data).split('\\n')[2][:-2])) # print out the response
        if site in allSamples: # if the site number is already a key in the dictionary...
            allSamples[site].append(float(str(input_data).split('\\n')[2][:-2])+100) # append the new measurement to that site list
        else: # if this is a new site and doesn't have a key yet...
            allSamples[site] = [float(str(input_data).split('\\n')[2][:-2])+100] # make a new key and give it the measurement value
        return allSamples # send us back the updated dictionary

### Collecting Data

Following the tape measure, walk along the dock and take a measurements at the sites spaced 1 meter apart.  Lower the transducer into the water to the tape mark indicating 10cm is at the surface.

Start by setting the site number (which is the same as the meter marker on the tape measure, i.e., 10 meters is site 10):

In [ ]:
site = 1 # Change this value for each site

Next, use the code cell below to collect a sample, and repeat to collect 5 samples at each site.  Do all of the values collected appear to be similar?  Do you think they are correct?

In [ ]:
allSamples = dist(triggerPin,echoPin,soundSpeed,site,allSamples)
print('Measurements for Site '+ str(site), allSamples[site])

We can look at the entire dictionary by running the cell below:

In [ ]:
print(allSamples)

In case you take a measurement by mistake and need to remove it, you can use `allSamples[`*Site Number*`].remove(`*Bad Value*`)` to remove it from the site key.

Once your sample table is complete, create a plot using the cell below.  Add a title and axis labels.

In [ ]:
plt.plot(list(allSamples.keys()),list(allSamples.values()),'.b')
plt.gca().invert_yaxis()

If you have time, try making a new plot using the data in your dictionary that shows the median of the measurements at each distance that you believe to be correct in order to remove noise.  